In [49]:
import os
import json
import numpy as np
from scipy.spatial.transform import Rotation as R

In [50]:
def euler_to_quaternion(euler_angles):
    """
    Convert Euler angles (rotation3D) to quaternion [qx, qy, qz, qw].
    :param euler_angles: List or tuple of Euler angles [x, y, z] in radians.
    :return: Quaternion [qx, qy, qz, qw]
    """
    rotation = R.from_euler('xyz', euler_angles)
    quaternion = rotation.as_quat()  # Returns [qx, qy, qz, qw]
    return quaternion

In [51]:
def convert_to_target_format(source_json, threshold=0.0):
    # Example label mapping from `labelId` to `detection_name`
    label_mapping = {0: "car", 1: "truck", 2: "bus", 3: "trailer", 4: "construction_vehicle", 5: "pedestrian", 6: "motorcycle", 7: "bicycle", 8: "traffic_cone", 9: "barrier"}
    
    target_data = []

    primary_token = str(source_json['sweepTimestamp'])
    
    for _, signal_data in source_json['signalsReceived'].items():
        for detection in signal_data['detections']:
            # Extracting and converting data
            translation = [
                detection['center3D']['x'], 
                detection['center3D']['y'], 
                detection['center3D']['z']
            ]
            
            size = [
                detection['size3D']['x'], 
                detection['size3D']['y'], 
                detection['size3D']['z']
            ]
            
            # Convert Euler angles to quaternion
            euler_angles = [
                detection['rotation3D']['x'],
                detection['rotation3D']['y'],
                detection['rotation3D']['z']
            ]
            rotation = euler_to_quaternion(euler_angles)  # Convert to quaternion
            
            detection_score = detection['confidence']
            detection_name = label_mapping.get(detection['labelId'], 'unknown')
            
            # Adding default velocity and num_pts (you can adjust these if you have the data)
            velocity = np.array([0.0, 0.0])
            
            # Creating the final target format dictionary
            target_format = {
                'sample_token': primary_token,
                'translation': translation,
                'size': size,
                'rotation': rotation.tolist(),  # Convert numpy array to list
                'velocity': velocity,
                'detection_name': detection_name,
                'detection_score': detection_score,
                'attribute_name': ''
            }

            if detection_score > threshold:
                target_data.append(target_format)
    
    return primary_token, target_data

In [52]:
# Helper function to convert numpy arrays to lists
def convert_numpy_to_list(data):
    if isinstance(data, np.ndarray):
        return data.tolist()
    elif isinstance(data, list):
        return [convert_numpy_to_list(item) for item in data]
    elif isinstance(data, dict):
        return {key: convert_numpy_to_list(value) for key, value in data.items()}
    return data

In [53]:
SOURCE_DIR = '/root/metrics/results'
SOURCE_DIR = '/root/metrics/converted-jsons/results_converted_05threshold'

threshold = 0.5

In [54]:
os.makedirs(SOURCE_DIR, exist_ok=True)

for file in os.listdir(SOURCE_DIR):
    # Read the source JSON file
    with open(os.path.join(SOURCE_DIR, file), 'r') as f:
        source_json = json.load(f)

    # Convert the data to the target format
    primary_token, converted_data = convert_to_target_format(source_json, threshold)

    # Convert any numpy arrays to lists so they can be JSON serializable
    converted_data = convert_numpy_to_list(converted_data)

    target_file = os.path.join(SOURCE_DIR, str(primary_token) + '.json')
    if not os.path.exists(target_file):
        target_data = {'results': {}}
    else:
        with open(target_file, 'r') as f:
            target_data = json.load(f)

    target_data['results'][primary_token] = converted_data

    # Add the "meta" field at the same level as "results"
    target_data['meta'] = {
        "use_camera": False,
        "use_lidar": True,
        "use_radar": False,
        "use_map": False,
        "use_external": False
    }

    # Write the target JSON file
    with open(target_file, 'w') as f:
        json.dump(target_data, f, indent=4)